# Escrevendo uma função de recomendação da Movie Lens

## Introdução

O dataset [MovieLens](https://grouplens.org/datasets/movielens/) é um conjunto de dados que contém várias classificações (ratings) de filmes por usuários. Este conjunto de dados é frequentemente usado para experimentos de pesquisa e prototipagem em sistemas de recomendação. O projeto MovieLens foi criado pelo GroupLens, um laboratório de pesquisa na University of Minnesota, e serve como uma das referências padrão na área de sistemas de recomendação.

O conjunto de dados tem várias versões, algumas contendo apenas algumas dezenas de milhares de classificações e outras contendo até 20 milhões ou mais. Cada registro geralmente contém:

    ID do usuário que deu a classificação
    ID do filme classificado
    A classificação dada (geralmente em uma escala de 1 a 5)
    Um atributo timestamp indicando quando a classificação foi dada

Algumas versões também incluem informações adicionais, como tags atribuídas aos filmes, gêneros, e até mesmo links para dados relacionados, como imagens de capas de filmes ou metadados. Por ser um conjunto de dados bem estruturado e extensivo, o MovieLens é amplamente utilizado para demonstrar algoritmos de recomendação, desde métodos simples, como a Filtragem Colaborativa e a Filtragem Baseada em Conteúdo, até técnicas mais avançadas como Sistemas de Recomendação baseados em Aprendizado Profundo.

## Baixando o dataset

In [ ]:
# No Colab não é necessário executar esse comando
# %pip install gdown

In [ ]:
# Importa o dataset de filmes
!gdown 1ovr90WWjeLh_PWqe5yLIK-1aIzHvtuDk

In [ ]:
# Import o dataset com as avaliações dos usuários
!gdown 1pN2Upg6J7mie1esMREFD5rHjZOpk9QXb

## Lendo o dataset para as recomendações

**Bibliotecas**

In [5]:
import pandas as pd
import numpy as np

### `movies.dat`

**Testando o acesso ao dataset**

In [6]:
# Adapte o nome do caminho para usar o caderno no Colab
df_filmes = pd.read_csv('./movies.dat', sep='::', engine='python', names=['id_filme', 'nome', 'categoria'])
df_filmes.head()

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### `ratings.csv`

**Testando o acesso ao dataset de ratings**

In [8]:
#  Adapte o nome do caminho para usar o caderno no Colab
df_ava = pd.read_csv(
        './ratings_mini.csv',
        sep= ',', # Separador
        dtype={'id_usuario': np.int32, 'id_filme': np.int32, 'avaliacao': np.float64, 'timestamp':np.int32})
df_ava.head()

,id_usuario,id_filme,avaliacao,timestamp
0,5,1,1.0,857911264
1,5,7,3.0,857911357
2,5,25,3.0,857911265
3,5,28,3.0,857913507
4,5,30,5.0,857911752


## Criando a função de recomendação

Para criar uma função de recomendação, temos que primeiro ler o modelo treinado e em seguida utilizar as informações do dataset para enriquecer a resposta.

#### Lendo o modelo treinado

In [1]:
from joblib import load
model = load('./movie_recommender_model.joblib')
model

#### Função de recomendação

In [9]:
def recommend_top_n_movies(user_id, model, df_ava, df_filmes, n_recomendacoes=5):
    # Encontrar todos os filmes únicos no dataset
    todos_filmes = df_ava['id_filme'].unique()
    
    # Encontrar os filmes que o usuário já avaliou
    filmes_avaliados = df_ava[df_ava['id_usuario'] == user_id]['id_filme'].tolist()
    
    # Filmes que o usuário não assistiu ainda
    filmes_nao_assistidos = [filme for filme in todos_filmes if filme not in filmes_avaliados]
    
    # Fazendo previsões para os filmes não assistidos
    predicoes = [model.predict(user_id, filme) for filme in filmes_nao_assistidos]
    
    # Ordenando as previsões em ordem decrescente (ou não crescente)
    predicoes_ordenadas = sorted(predicoes, key=lambda x: x.est, reverse=True)
    
    # Retornar apenas as top 'n_recomendacoes' recomendações
    top_preds = predicoes_ordenadas[:n_recomendacoes]
    
    # Lista de retorno
    recomendacoes = []
    for pred in top_preds:
        nome_filme = df_filmes[df_filmes['id_filme'] == pred.iid]['nome'].iloc[0]
        # Armazena as informações em um formato de dicionário que poderia ser útil para converter para JSON
        recomendacoes.append({"id_filme": pred.iid, "nome": nome_filme, "avaliacao_estimada": pred.est})

    return recomendacoes

# Uso da função
user_id = 7
recommend_top_n_movies(user_id, model, df_ava, df_filmes)

[{'id_filme': 1945,
  'nome': 'On the Waterfront (1954)',
  'avaliacao_estimada': 4.803876380878684},
 {'id_filme': 926,
  'nome': 'All About Eve (1950)',
  'avaliacao_estimada': 4.798729743644757},
 {'id_filme': 922,
  'nome': 'Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)',
  'avaliacao_estimada': 4.766683067787066},
 {'id_filme': 1230,
  'nome': 'Annie Hall (1977)',
  'avaliacao_estimada': 4.726328723459158},
 {'id_filme': 858,
  'nome': 'Godfather, The (1972)',
  'avaliacao_estimada': 4.725264643467319}]